## Command to wait for database connection

 > 1) **Before we run the server**, we have to **check if the database is connected** in the docker-compose file.
 
 > 2) To do this, we need something that can be executed from terminal. We can make something like this work,
 ```bash
 python manage.py wait_for_db
 python manage.py runserver
 ```

> 3) Before writing this function, in TDD, **we first have to find out that part of the code that connects to the database**

>4) The connected database object can be obtained as

```python
        from django.db import connections
        # get the database with keyword 'default' from settings.py
        db_conn = connections['default']
```
>> When this database is not available **OperationalError** is raised

> 5) Sometimes databases can take long time to connect. We may not want to actually connect to a database while testing. In such cases, we can **mock a database connection**

> 6) To do this, we ll have to dig into the source code to find out what happens when **connections['default']** is called,

>> The **connections** attribute is defined in **Django/db/\__init\__.py** as
```bash
connections = ConnectionHandler()
```
>> **connections['default']** calls the **\__getitem\__** method in **ConnectionHandler** class(advanced python concepts). In this case, we see that the \__getitem\__ method has been redefined in the **ConnectionHandler** class

> 7) Hence, we can create a patch like this in our test case

<div style="border:2px solid;margin:0 20px 0 20px" ></div> 

```python
from unittest.mock import patch
from django.core.management import call_command
from django.db.utils import OperationalError
from django.test import TestCase


class CommandTests(TestCase):
    """ we create a command wait_for_db to check if database is
    available before running server. Here we test if this command"""

    # check if the command works when db is available
    def test_wait_for_db_ready(self):
        with patch('django.db.utils.ConnectionHandler.__getitem__') as gi:
            gi.return_value = True
            call_command('wait_for_db')
            self.assertEqual(gi.call_count, 1)

    # check if command works when db is not connected
    # by default, we ll have sleep function to wait for 1s if database is not available
    # we dont want this sleep to wait for this test
    @patch('time.sleep', return_value=True)
    def test_wait_for_db(self, ts):
        with patch('django.db.utils.ConnectionHandler.__getitem__') as gi:
            # make the patch return error for first five calls
            # and return true on sixth call
            gi.side_effect = [OperationalError]*5 + [True]
            call_command('wait_for_db')
            self.assertEqual(gi.call_count, 6)
```
<div style="border:2px solid;margin:0 20px 0 20px" ></div> 


### wait_for_db command 

Refer Django custom commands notes 

> 1) Create a file **core/management/commands/wait_for_db.py**

> 2) Add **\__init\__.py** in the folders **management** and **commands** if it is not already available

<div style="border:2px solid;margin:0 20px 0 20px" ></div> 

```python
from django.core.management.base import BaseCommand
from django.db import connections
from django.db.utils import OperationalError
import time


class Command(BaseCommand):
    """ Django command to pause execution until database is available"""
    def handle(self, *args, **kwargs):
        self.stdout.write('waiting for db ...')
        db_conn = None
        while not db_conn:
            try:
                # get the database with keyword 'default' from settings.py
                db_conn = connections['default']
                # prints success messge in green
                self.stdout.write(self.style.SUCCESS('db available'))
            except OperationalError:
                self.stdout.write("Database unavailable, waiting 1 second ...")
                time.sleep(1)
```
<div style="border:2px solid;margin:0 20px 0 20px" ></div> 


### Add command in docker-compose

> Change the docker-compose command for **app** service as follows

```yml
    command: >
      sh -c "python manage.py wait_for_db &&
             python manage.py migrate &&
             python manage.py runserver 0.0.0.0:8000"
```


> To run the server, hit
```bash
docker-compose up
```
> If super user was not already created, hit this command in a separate terminal and create a super user
```bash
docker-compose run app sh
```